## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-30-18-57-00 +0000,wsj,Fed Meeting Today: Federal Reserve Leaves Rate...,https://www.wsj.com/livecoverage/fed-meeting-i...
1,2025-07-30-18-55-00 +0000,wsj,The U.S. economy returned to growth in the sec...,https://www.wsj.com/economy/us-gdp-q2-2025-359...
2,2025-07-30-18-54-47 +0000,startribune,Live: Cyberattacks like the one in St. Paul co...,https://www.startribune.com/st-paul-mn-cyber-a...
3,2025-07-30-18-54-00 +0000,wsj,Opinion | Who Is Going to Miss NPR?,https://www.wsj.com/opinion/who-is-going-to-mi...
4,2025-07-30-18-51-00 +0000,wsj,Opinion | Macron’s View of Palestine Fits the ...,https://www.wsj.com/opinion/macrons-view-of-pa...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2319/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
42,trump,53
115,tsunami,24
138,new,18
217,palestinian,17
84,gaza,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
226,2025-07-30-01-39-32 +0000,nypost,Senate confirms Trump lawyer Emil Bove for fed...,https://nypost.com/2025/07/29/us-news/senate-c...,113
117,2025-07-30-14-30-00 +0000,wsj,The Senate confirmed former Trump defense atto...,https://www.wsj.com/politics/policy/trumps-for...,109
277,2025-07-29-22-00-59 +0000,nyt,"Trump Says Gaza Will Get More Aid, but Israel ...",https://www.nytimes.com/2025/07/29/us/politics...,104
221,2025-07-30-02-05-07 +0000,cbc,U.S. Senate confirms former Trump lawyer Emil ...,https://www.cbc.ca/news/world/trump-lawyer-emi...,103
285,2025-07-29-21-42-41 +0000,nypost,Top Zelensky official praises Trump for giving...,https://nypost.com/2025/07/29/us-news/top-zele...,101


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
226,113,2025-07-30-01-39-32 +0000,nypost,Senate confirms Trump lawyer Emil Bove for fed...,https://nypost.com/2025/07/29/us-news/senate-c...
136,72,2025-07-30-13-13-39 +0000,bbc,Tsunami warning sparks evacuations in Japan an...,https://www.bbc.com/news/articles/c987dwrdl0zo...
323,65,2025-07-29-19-52-56 +0000,nyt,UK Will Recognize Palestinian Statehood In Sep...,https://www.nytimes.com/2025/07/29/world/europ...
251,55,2025-07-29-23-29-20 +0000,nypost,Zohran Mamdani’s NYC election rivals have litt...,https://nypost.com/2025/07/29/us-news/zohran-m...
87,53,2025-07-30-16-06-00 +0000,wsj,Any further Bank of Canada rate reduction depe...,https://www.wsj.com/articles/bank-of-canada-st...
0,51,2025-07-30-18-57-00 +0000,wsj,Fed Meeting Today: Federal Reserve Leaves Rate...,https://www.wsj.com/livecoverage/fed-meeting-i...
185,43,2025-07-30-09-30-00 +0000,wsj,The shake-up for charitable giving inside the ...,https://www.wsj.com/politics/policy/trump-tax-...
292,41,2025-07-29-21-30-24 +0000,nypost,Ukrainian mom-to-be expecting first child afte...,https://nypost.com/2025/07/29/world-news/ukrai...
95,40,2025-07-30-15-56-06 +0000,nyt,Israeli Far-Right Minister Promotes Plan for J...,https://www.nytimes.com/2025/07/29/world/middl...
15,39,2025-07-30-18-31-33 +0000,wapo,NTSB hearing live updates: Army helicopter cre...,https://www.washingtonpost.com/business/2025/0...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
